### Build balanced data

pos:neg = 1:1

In [1]:
from collections import defaultdict
train_data = defaultdict(list)
dev_data = defaultdict(list)
test_data = defaultdict(list)

with open('train_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        train_data[label].append(sent)
with open('dev_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        dev_data[label].append(sent)
with open('test_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        test_data[label].append(sent)

In [3]:
len_neg_train = len(train_data['0'])
train_data['1'] = train_data['1'][:len_neg_train]

len_neg_dev = len(dev_data['0'])
dev_data['1'] = dev_data['1'][:len_neg_dev]

len_neg_test = len(test_data['0'])
test_data['1'] = test_data['1'][:len_neg_test]

In [5]:
import random
b_train_data = []
for label, sents in train_data.items():
    for s in sents:
        b_train_data.append((s, label))
random.shuffle(b_train_data)

b_dev_data = []
for label, sents in dev_data.items():
    for s in sents:
        b_dev_data.append((s, label))
random.shuffle(b_dev_data)

b_test_data = []
for label, sents in test_data.items():
    for s in sents:
        b_test_data.append((s, label))
random.shuffle(b_test_data)

In [6]:
with open('balance_train.tsv', 'w') as f:
    for d in b_train_data:
        f.write(d[0] + '\t' + d[1] + '\n')
        
with open('balance_dev.tsv', 'w') as f:
    for d in b_dev_data:
        f.write(d[0] + '\t' + d[1] + '\n')
        
with open('balance_test.tsv', 'w') as f:
    for d in b_test_data:
        f.write(d[0] + '\t' + d[1] + '\n')

In [7]:
print(len(b_train_data))
print(len(b_dev_data))
print(len(b_test_data))

129310
27502
27600


### Fit logistic regression model

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = []
labels = []
with open('balance_train.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        corpus.append(sent)
        labels.append(int(label))
        
test_corpus = []
test_labels = []
with open('balance_test.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        test_corpus.append(sent)
        test_labels.append(int(label))

In [9]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus+test_corpus)

In [10]:
train_X = X[:len(corpus)]
test_X = X[len(corpus):]

In [11]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(train_X, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
pred = clf.predict(test_X)